In [38]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [39]:
data = pd.read_csv("data\\flights_train.csv")
testData = pd.read_csv("data\\flights_test.csv")
airportData = pd.read_csv('data\\airports.csv', low_memory=False)

In [40]:
testModelData = data
# Add departure hour
testModelData['SCHEDULED_DEPARTURE'] = testModelData['SCHEDULED_DEPARTURE'].apply('{:0>4}'.format)
testModelData['DEPARTURE_HOUR'] = testModelData['SCHEDULED_DEPARTURE'].str[:2]
# Add arrival hour
testModelData['SCHEDULED_ARRIVAL'] = testModelData['SCHEDULED_ARRIVAL'].apply('{:0>4}'.format)
testModelData['ARRIVAL_HOUR'] = testModelData['SCHEDULED_ARRIVAL'].str[:2]
testModelData['DAY_OF_MONTH'] = testModelData['MONTH']*100 + testModelData['DAY']
testModelData.head

<bound method NDFrame.head of               id  YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER  \
0              0  2015      1    1            4      AS             98   
1              1  2015      1    1            4      AA           2336   
2              2  2015      1    1            4      US            840   
3              3  2015      1    1            4      AA            258   
4              4  2015      1    1            4      AS            135   
...          ...   ...    ...  ...          ...     ...            ...   
2818548  2818548  2015      6   30            2      UA           1180   
2818549  2818549  2015      6   30            2      UA           1192   
2818550  2818550  2015      6   30            2      UA           1480   
2818551  2818551  2015      6   30            2      UA           1637   
2818552  2818552  2015      6   30            2      UA           1903   

        TAIL_NUMBER ORIGIN_AIRPORT DESTINATION_AIRPORT  ... WHEELS_OFF  \
0      

In [41]:
# create dummies
dummy_fields = ['ARRIVAL_HOUR', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE']
for each in dummy_fields:
    dummies = pd.get_dummies(testModelData[each], prefix=each, drop_first=False)
    testModelData = pd.concat([testModelData, dummies], axis=1)
testModelData.head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
0,0,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,0,0,0,0,0,0,0,0,0,0
1,1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,0,0,0,0,0,0,0,0,0,0
2,2,2015,1,1,4,US,840,N171US,SFO,CLT,...,0,0,0,0,0,0,0,1,0,0
3,3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,0,0,0,0,0,0,0,0,0,0
4,4,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# remove original columns
testModelData = testModelData.drop(dummy_fields, axis=1)

In [43]:
# Select columns
trainData3 = testModelData.drop(columns=['YEAR', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DEPARTURE_TIME', 'TAXI_OUT','AIR_TIME', 'DISTANCE'])

In [44]:
# ReduceDataSet

# Split data
X = trainData3.drop(columns='ARRIVAL_DELAY')
Y = trainData3['ARRIVAL_DELAY']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(2254842, 694) (563711, 694) (2254842,) (563711,)


In [45]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0, n_jobs=-1)
regressor.fit(X_train, Y_train)
# R^2 scores
print('Train:', regressor.score(X_train, Y_train))
print('Test:', regressor.score(X_test, Y_test))

MemoryError: Unable to allocate 11.5 GiB for an array with shape (683, 2254842) and data type object

# TEST DATA

In [ ]:

# Add departure hour
testData['SCHEDULED_DEPARTURE'] = testData['SCHEDULED_DEPARTURE'].apply('{:0>4}'.format)
testData['DEPARTURE_HOUR'] = testData['SCHEDULED_DEPARTURE'].str[:2]
# Add arrival hour
testData['SCHEDULED_ARRIVAL'] = testData['SCHEDULED_ARRIVAL'].apply('{:0>4}'.format)
testData['ARRIVAL_HOUR'] = testData['SCHEDULED_ARRIVAL'].str[:2]

In [ ]:
# Create dummies
dummy_fields = ['ARRIVAL_HOUR', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE']
for each in dummy_fields:
    dummies = pd.get_dummies(testData[each], prefix=each, drop_first=False)
    testData = pd.concat([testData, dummies], axis=1)
testData.head()   

In [ ]:
# remove original dummies
testData = testData.drop(dummy_fields, axis=1)


In [ ]:
# Select data
testData = testData.drop(columns=['YEAR', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DEPARTURE_TIME', 'TAXI_OUT','AIR_TIME', 'DISTANCE'])
#testData = testData.drop(columns=['TAIL_NUMBER', 'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_HOUR']) #megiel added 
# select missing columns
missingCols = X_test.columns.difference(testData.columns)
# add zeros to missing columns
for col in missingCols:
    testData[col] = 0

In [ ]:
finalPrediction = regressor.predict(testData) # model features 693
finalDataset = pd.DataFrame(data=finalPrediction, columns=['ARRIVAL_DELAY'])

In [ ]:
# save file
finalDataset.to_csv('prediction.csv', index_label='id')